In [1]:
import pandas as pd
import Levenshtein

In [2]:
words = pd.read_csv('words.csv')

In [3]:
words.head()

,Français,Hiragana,Kanji
0,enseignant,きょうし,教室
1,étudiant,がくせい,学生
2,employé d'une compagnie,かいしゃいん,会社員
3,employé (avec nom de compagnie),しゃいん,社員
4,employé de banque,ぎんこういん,銀行員


In [4]:
shuffled_words = words.sample(frac=1).reset_index(drop=True)

In [5]:
def is_similar_japanese_answer(user_answer, correct_answer):
    similarity_threshold = 0.8
    distance = Levenshtein.distance(user_answer.lower(), correct_answer.lower())
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    return similarity >= similarity_threshold

In [6]:
def is_similar_french_answer(user_answer, correct_answer):
    similarity_threshold = 0.5
    if pd.isna(user_answer) or pd.isna(correct_answer):
        return False
    user_answer = user_answer.lower()
    correct_answer = correct_answer.lower()
    substring_match = user_answer in correct_answer or correct_answer in user_answer
    if substring_match:
        return True
    distance = Levenshtein.distance(user_answer, correct_answer)
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    
    return similarity >= similarity_threshold

In [7]:
shuffled_sample = shuffled_words.sample(n=20)
shuffled_sample

,Français,Hiragana,Kanji
45,médecin,いしゃ,医者
136,Voulez-vous en prendre un autre [~]?,もう一杯いかがですか,NaN
75,distributeur automatique,じどうはんばいき,自動販売機
87,consulter / chercher,しらべます,調べます
16,montant/somme d'argent,きんがく,金額
56,journal intime,にっき,日記
62,prière,おいのり,お祈り
123,visiter un lieu pour études,けんがくします,見学します
91,baseball,やきゅう,野球
76,bureau,じむしょ,事務所


In [8]:
seen_fr = []
correct_count = 0
score_fr = 0
shuffled_sample_jp = shuffled_sample.sample(frac=1).reset_index(drop=True)

for i in range(len(shuffled_sample_jp)):
    if shuffled_sample_jp['Français'].iloc[i] not in seen_fr:
        ask_h = input(f"Comment dire {shuffled_sample_jp['Français'].iloc[i]} en hiragana ?")
        seen_fr.append(shuffled_sample_jp['Français'].iloc[i])

        if is_similar_japanese_answer(ask_h, shuffled_sample_jp['Hiragana'].iloc[i]):
            kanji_value = shuffled_sample_jp['Kanji'].iloc[i]
            if pd.notna(kanji_value):
                ask_k = input("Et en kanji ?")
                if is_similar_japanese_answer(ask_k, kanji_value):
                    correct_count += 1
                    score_fr += 1
                    print('Correct !')
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
                else:
                    print(f"Incorrect ! La bonne réponse est : {kanji_value}")
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
            else:
                correct_count += 1
                score_fr += 1
                print('Correct !')
                print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
        else:
            print(f"Incorrect ! La bonne réponse est : {shuffled_sample_jp['Hiragana'].iloc[i]}")
            print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')

if correct_count == len(shuffled_sample_jp):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Comment dire distributeur automatique en hiragana ?
Incorrect ! La bonne réponse est : じどうはんばいき
Bonnes réponses : 0/20
Comment dire réunion en hiragana ?かいぎ
Et en kanji ?会議
Correct !
Bonnes réponses : 1/20
Comment dire préfecture en hiragana ?けん
Et en kanji ?県
Correct !
Bonnes réponses : 2/20
Comment dire consulter / chercher en hiragana ?さがします
Incorrect ! La bonne réponse est : しらべます
Bonnes réponses : 2/20
Comment dire bureau en hiragana ?じむしょ
Et en kanji ?事務所
Correct !
Bonnes réponses : 3/20
Comment dire animé / fréquenté en hiragana ?にぎやか
Correct !
Bonnes réponses : 4/20
Comment dire baseball en hiragana ?やきゅう
Et en kanji ?野球
Correct !
Bonnes réponses : 5/20
Comment dire journal intime en hiragana ?
Incorrect ! La bonne réponse est : にっき
Bonnes réponses : 5/20
Comment dire saison en hiragana ?きせつ
Et en kanji ?季節
Correct !
Bonnes réponses : 6/20
Comment dire visiter un lieu pour études en hiragana ?りゅうがく
Incorrect ! La bonne réponse est : けんがくします
Bonnes réponses : 6/20
Comment dire V

In [ ]:
seen_k=[]
correct_count = 0
score_jp = 0
shuffled_sample_fr = shuffled_sample.sample(frac=1).reset_index(drop=True)
for i in range(len(shuffled_sample_fr)):
    kanji = shuffled_sample_fr['Kanji'].iloc[i]
    if pd.notna(kanji) and kanji not in seen_k:
        ask_h = input(f"Que veut dire ce kanji : {kanji}?")
        if is_similar_french_answer(ask_h, shuffled_sample_fr['Français'].iloc[i]):
            seen_k.append(kanji)
            correct_count += 1
            score_jp += 1
            print(f"Correct ! C'était bien {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')
        else:
            print(f"Incorrecte ! La bonne réponse était : {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')

if correct_count == len(shuffled_sample_fr):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Que veut dire ce kanji : 修理します?réparer
Correct ! C'était bien réparer
Bonnes réponses : 1/20
Que veut dire ce kanji : 封筒?enclose
Correct ! C'était bien enveloppe
Bonnes réponses : 2/20
Que veut dire ce kanji : 見学します?faire de la rechercher
Incorrecte ! La bonne réponse était : visiter un lieu pour études
Bonnes réponses : 2/20
Que veut dire ce kanji : 季節?saison
Correct ! C'était bien saison
Bonnes réponses : 3/20
Que veut dire ce kanji : 金額?somme argent
Correct ! C'était bien montant/somme d'argent
Bonnes réponses : 4/20
Que veut dire ce kanji : 医者?médecin
Correct ! C'était bien médecin
Bonnes réponses : 5/20
Que veut dire ce kanji : 研究します?faire de la rechercher
Correct ! C'était bien faire de la recherche
Bonnes réponses : 6/20
Que veut dire ce kanji : 野球?baseball
Correct ! C'était bien baseball
Bonnes réponses : 7/20
Que veut dire ce kanji : 低い?bas
Correct ! C'était bien bas
Bonnes réponses : 8/20
Que veut dire ce kanji : 自動販売機?distributeur automatique
Correct ! C'était bien distribut